# Airbnb Seattle Dataset

The questions that should be reviewed in this project are: <br>

    1. What are the main rental price drivers?
    2. What are the price differences between accomodations with varying prices business days to weekends?
    3. Are there any clear differences between accomodations with constant and varying prices. 

# Data Understanding

## Libraries for complete notebook

In [31]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import available dataset

In [32]:
path_work = 'C:/Users/matze/Documents/workspace/udc_DS/airbnb'
filenames = ['calendar', 'listings', 'reviews']


def read_csv(name, error_bad_lines=True):
    with open(os.path.join(path_work, name + '.csv'), 'r', errors='replace') as source:
        df = pd.read_csv(source, header=0, parse_dates=True,
                         dayfirst=True)#, error_bad_lines=False)
    return df

# reviews.csv data is not required for analysis
calendar = read_csv('calendar')
listings = read_csv('listings')
#listings

### Data preparation

In [33]:
#check calendar dtypes
calendar.dtypes

listing_id     int64
date          object
available     object
price         object
dtype: object

In [34]:
# convert calendar dtypes
calendar['price'] = calendar['price'].map(
    lambda x: x.lstrip('$'), na_action='ignore')
calendar['price'] = calendar['price'].map(
    lambda x: x.replace(',', ''), na_action='ignore')
calendar['price'] = pd.to_numeric(calendar['price'])

calendar['date'] = pd.to_datetime(calendar['date'])

calendar['available'] = calendar['available'].map({'t': True, 'f': False})
# calendar.dtypes

In [35]:
# Listings data prep
# Check for columns with np.nan only and drop
col_nan = listings.isna().all()[listings.isna().all() == True].index
listings = listings.drop(columns=col_nan)

# Drop url's as almost all unique hence no value
listings = listings.drop(
    columns=[col for col in listings.columns if col[-4:] == '_url'])

# Remove text columns not categorial or with many cats
# assumption: eg streets to detailed, cleansed neighboorhoods kept
listings = listings.drop(columns=['name', 'summary', 'space', 'description',
                                  'neighborhood_overview', 'notes', 'transit',
                                  'host_name', 'host_location', 'host_about',
                                  'host_verifications', 'street', 'country_code',
                                  'country', 'first_review', 'last_review',
                                  'neighbourhood'])

# convert objects into usable dtypes
col_obj = listings.select_dtypes('object').columns
for col in col_obj:
    # print(col)
    # Remove $ and % sign in prep to change cols to numeric dtype
    listings[col] = listings[col].map(
        lambda x: x.lstrip('$'), na_action='ignore')
    listings[col] = listings[col].map(
        lambda x: x.rstrip('%'), na_action='ignore')

    # change to numeric dtype; remove thousands separator first
    if col in ['price', 'weekly_price', 'monthly_price', 'security_deposit',
               'cleaning_fee', 'extra_people', 'host_response_rate',
               'host_acceptance_rate']:
        listings[col] = listings[col].map(
            lambda x: x.replace(',', ''), na_action='ignore')
        listings[col] = pd.to_numeric(listings[col])

    # ignore rows that start with np.nan for the moment
    elif listings[col][0] != listings[col][0]:
        listings[col] = listings[col]

    # covert to datetimes
    elif re.search('\d\d\d\d-\d\d-\d\d', listings[col][0]):
        listings[col] = pd.to_datetime(listings[col])

    # convert to boolean as only t and f
    elif re.search('^[t|f]$', listings[col][0]):
        listings[col] = listings[col].replace({'t': True, 'f': False})

    # keep all others
    else:
        listings[col] = listings[col]

In [37]:
#listings.describe(include='all',datetime_is_numeric=True)

In [43]:
# Amenities to be hot encoded separately as one str for all entries
# Split into categories and remove/clean
pat = re.compile('[{}"]')
df_amt = listings['amenities'].apply(lambda x: sorted(
    [re.sub(pat, '', y) for y in x.split(',')]))

# Set of all possible amenities
amenities = []
for each in df_amt:
    amenities = amenities + each
amenities = set(amenities)
amenities.remove('')

# df hot encoded amenities
df_amt_hot = pd.DataFrame(columns=sorted(amenities), index=listings.index)

for row in range(len(listings.index)):
    for col, each in enumerate(sorted(amenities)):
        if each in df_amt[row]:
            df_amt_hot.iloc[row, col] = True
        else:
            df_amt_hot.iloc[row, col] = False

# Replace amenities with hot encoded amenities inlistings
listings = pd.concat([listings, df_amt_hot], axis=1)
listings = listings.drop(columns=['amenities'])

In [45]:
#listings

In [49]:
# Generate listings with hot encoding
listings_dum = pd.get_dummies(listings, drop_first=True)
#listings_dum